In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import csv

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

from dataset import get_cifar10_dataset
from train_val_test import train_val, test
from model import Net

import argparse

# 학습 로그를 기록하고 CSV로 저장하는 함수
def train_val(model, device, num_epochs, train_loader, val_loader, criterion, optimizer, scheduler=None):
    torch.multiprocessing.freeze_support()
    model.to(device)

    train_accuracies = []
    val_accuracies = []
    logs = []  # To store logs
    best_val_accuracy = 0.0  # Best validation accuracy to track improvement

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        train_accuracy = 100. * correct / total
        train_accuracies.append(train_accuracy)
        print(f'Epoch {epoch+1}, Train Loss: {epoch_loss}, Train Accuracy: {train_accuracy:.2f}%')

        if scheduler is not None:
            scheduler.step()

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                outputs = model(inputs)

                loss = criterion(outputs, targets)

                val_running_loss += loss.item() * inputs.size(0)

                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

            val_epoch_loss = val_running_loss / len(val_loader.dataset)
            val_accuracy = 100. * correct / total
            val_accuracies.append(val_accuracy)
            print(f'Epoch {epoch+1}, Validation Loss: {val_epoch_loss}, Validation Accuracy: {val_accuracy:.2f}%')

            # Save the model if the validation accuracy is the best we've seen so far.
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy

        # Log the results
        logs.append({
            'epoch': epoch + 1,
            'train_loss': epoch_loss,
            'train_accuracy': train_accuracy,
            'val_loss': val_epoch_loss,
            'val_accuracy': val_accuracy,
            'test_accuracy': None  # Placeholder, will be updated after test phase
        })

    return train_accuracies, val_accuracies, logs

# 테스트 함수
def test(model, device, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    test_accuracy = 100 * correct / total
    print(f'Accuracy of the network on the 10000 test images: {test_accuracy:.2f}%')
    return test_accuracy

# 학습 및 검증 로그 기록 및 시각화
parser = argparse.ArgumentParser()
parser.add_argument('-s', '--lr_scheduler', default=None, help='Select learning rate scheduler (step, exponential, polynomial, cosine, cosinewarmup)')
args = parser.parse_args([])  # 빈 리스트를 사용하여 Jupyter Notebook에서 실행 가능하도록 설정

random_seed = 42

generator = torch.Generator().manual_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]
)

model = Net()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 200

dataset = get_cifar10_dataset(root='./data', train=True, download=False, transform=transform)
test_set = get_cifar10_dataset(root='./data', train=False, download=False, transform=transform)

train_set, val_set = random_split(dataset, [int(0.8 * len(dataset)), int(0.2 * len(dataset))], generator=generator)

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

criterion = nn.CrossEntropyLoss()

if args.lr_scheduler == 'step':
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
elif args.lr_scheduler == 'exponential':
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
elif args.lr_scheduler == 'polynomial':
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=num_epochs, power=2)
elif args.lr_scheduler == 'cosine':
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
elif args.lr_scheduler == 'cosinewarmup':
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20)
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = None

train_accuracies, val_accuracies, logs = train_val(model, device, num_epochs, train_loader, val_loader, criterion, optimizer, scheduler)
test_accuracy = test(model, device, test_loader)

# Update test accuracy in logs
for log in logs:
    log['test_accuracy'] = test_accuracy

# Save logs to CSV
keys = logs[0].keys()
with open('training_logs.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(logs)

# Extract values for plotting
epochs = range(1, num_epochs + 1)
train_losses = [log['train_loss'] for log in logs]
val_losses = [log['val_loss'] for log in logs]
train_accuracies = [log['train_accuracy'] for log in logs]
val_accuracies = [log['val_accuracy'] for log in logs]

# Plotting
plt.figure(figsize=(10, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'b', label='Train Loss')
plt.plot(epochs, val_losses, 'r', label='Validation Loss')
plt.title('Train and Validation Loss per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'b', label='Train Accuracy')
plt.plot(epochs, val_accuracies, 'r', label='Validation Accuracy')
plt.title('Train and Validation Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# CSV 파일로부터 데이터 읽기 및 그래프 그리기

import matplotlib.pyplot as plt
import csv

# CSV 파일로부터 데이터 읽기
def read_logs(csv_file):
    epochs = []
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    test_accuracies = []

    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            epochs.append(int(row['epoch']))
            train_losses.append(float(row['train_loss']))
            val_losses.append(float(row['val_loss']))
            train_accuracies.append(float(row['train_accuracy']))
            val_accuracies.append(float(row['val_accuracy']))
            test_accuracies.append(float(row['test_accuracy']))

    return epochs, train_losses, val_losses, train_accuracies, val_accuracies, test_accuracies

# 그래프 그리기
def plot_logs(epochs, train_losses, val_losses, train_accuracies, val_accuracies):
    plt.figure(figsize=(10, 5))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b', label='Train Loss')
    plt.plot(epochs, val_losses, 'r', label='Validation Loss')
    plt.title('Train and Validation Loss per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, 'b', label='Train Accuracy')
    plt.plot(epochs, val_accuracies, 'r', label='Validation Accuracy')
    plt.title('Train and Validation Accuracy per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# CSV 파일 경로
csv_file = 'training_logs.csv'

# CSV로부터 데이터 읽기
epochs, train_losses, val_losses, train_accuracies, val_accuracies, test_accuracies = read_logs(csv_file)

# 그래프 그리기
plot_logs(epochs, train_losses, val_losses, train_accuracies, val_accuracies)
